<a href="https://colab.research.google.com/github/kutyadog/ai_notebooks/blob/main/linkboxes_test_cj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using AI tools to help Journalism - Nov 2023

This is an exploration of harnasssing AI tools to automatically build (requires humans to launch/publish) elements on the website that will drive traffic.

* Links to summilar content (linkbox)
* Summaries

# Start it up

In [ ]:
!pip install -q gradio==3.41.0
# !pip install -q gradio
# !pip install https://gradio-builds.s3.amazonaws.com/fbcd9a9acef733f106b5138a61fce2190c12c76c/gradio-3.45.1-py3-none-any.whl
!pip install beautifulsoup4
# !pip install rake-nltk
!pip install keybert
!pip install -U textblob
!python -m textblob.download_corpora
!pip install -q openai
!pip install pytrends
from google.colab import userdata


import gradio
import requests
import pandas as pd
import math
import json
from bs4 import BeautifulSoup
from keybert import KeyBERT
import IPython
import openai
openai.api_key = userdata.get('OPENAI_API_KEY')
# import nltk; nltk.download('popular')

In [ ]:
# @title Story Utility functions

def get_story_headline(element):
  try:
    item = element['headlines']['basic']
  except KeyError:
    item = ''
  return item

def get_story_title(element):
  try:
    item = element['headlines']['meta_title'] # + " - The Washington Post"
  except KeyError:
    item = ''
  return item

def get_story_description(element):
  try:
    item = element['description']['basic']
  except KeyError:
    item = ''
  return item

def get_id(element):
  try:
    item = element['_id']
  except KeyError:
    item = ''
  return item

def get_content_elements(element):
  try:
    item = element['content_elements']
  except KeyError:
    item = ''
  return item

def get_story_section_path(element):
  try:
    item = element['taxonomy']['primary_section']['path']
  except KeyError:
    item = ''

  if item == '':
    try:
      item = element['additional_properties']['commercial_node']
    except KeyError:
      item = ''

  if item == '':
    try:
      item = element['taxonomy']['primary_site']['_id']
    except KeyError:
      item = ''

  return item

def get_story_canonical_url(element):
  try:
    item = element['headlines']['url']
  except KeyError:
    item = ''

  if item == '':
    try:
      item = element['canonical_url']
    except KeyError:
      item = ''

  return item

def get_story_publish_date(element):
  try:
    item = element['publish_date']
  except KeyError:
    item = ''
  return item

def get_story_slug(element):
  try:
    item = element['slug']
  except KeyError:
    item = ''
  return item

def get_story_type(element):
  try:
    item = element['type']
  except KeyError:
    item = ''
  return item

def get_story_subtype(element):
  try:
    item = element['subtype']
  except KeyError:
    item = ''
  return item

def get_tags(story_object):
  ignore_list = ['story-has-edit-branch', 'methode-draft', 'audio-article', 'coverage-active']
  xtags = []
  for tag in story_object['taxonomy']['tags']:
    if tag['slug'] not in ignore_list:
      if not tag['slug'].startswith('created-from-template-id-'):
        xtags.append(tag['slug'])
        # print(tag['slug'])
  return xtags

# pretty print JSON
# print(json.dumps(score_object, indent=2))

# headline = get_story_headline( story_object )
# meta_title = get_story_title( story_object )
# get_story_description( story_object )
# get_content_elements(story_object)
# get_story_section_path( story_object)
# get_story_canonical_url( story_object)

Types of link boxes:
*   SUBJECT: What To Know
*   More on SUBJECT
*   More from AUTHOR
*   More coverage: SUBJECT
*   SUBJECT (followed by several SUBTITLE: More info)
*   Previous coverage of SUBJECT



**Summarizing:**

*   Who is involved?
*   What happened?
*   When did it happen?
*   Where did it happen?
*   Why did it happen?





In [ ]:
# @title Prism functions
from google.colab import userdata

wapo_prism_key = userdata.get('WAPO_PRISM_KEY')

# Url decoder: https://www.urldecoder.org/
def get_prism_stories_by_tag(tag):

  fullpath = f"https://prism.ext.nile.works/content/v4/search/published?q=+taxonomy.tags.slug:%22{tag}%22&nocache=true&s=date&from=0&t=story&size=30&sort=display_date:desc&website=washpost&key={wapo_prism_key}&_sourceInclude=headlines.basic,headlines.meta_title,headlines.url,canonical_url,content_elements.type,content_elements.content,description.basic,publish_date,slug,type,subtype,taxonomy.primary_section.path,taxonomy.primary_section._id,website_url,taxonomy.tags.slug"
  print(fullpath)
  response = requests.get(fullpath)
  my_json_object = json.loads(response.content)
  return my_json_object

def get_prism_data_from_url(url):
  fixed_url = url.replace('https://www.washingtonpost.com', '')
  fixed_url = fixed_url.replace('https://washingtonpost.com', '')
  if fixed_url[-1] != '/' and not fixed_url.endswith("_story.html") :
    fixed_url += '/'
  fullpath = f"https://prism.ext.nile.works/content/v4?website=washpost&key={wapo_prism_key}&website_url={fixed_url}&website=washpost"
  response = requests.get(fullpath)
  my_json_object = json.loads(response.content)
  return my_json_object

# story_object = get_prism_data_from_url("https://www.washingtonpost.com/sports/2023/10/01/taylor-swift-kelce-chiefs-jets/")

# live-reporter-insight
# live-update
# live-all

# CHRIS THIS IS THE NEW PATH WITH ONLY FIELDS YOU WANT: (OF COURSE CHANGE THE TOP DOMAIN)
# https://prism.wpit.nile.works/content/v4/search/?website=washpost&sort=last_updated_date:desc&from=0&size=20&body=%7B%22query%22:%7B%22bool%22:%7B%22must%22:%5B%7B%22term%22:%7B%22type%22:%22story%22%7D%7D,%7B%22match_phrase_prefix%22:%7B%22slug%22:%22linkbox%22%7D%7D,%7B%22term%22:%7B%22revision.published%22:%22true%22%7D%7D,%7B%22range%22:%7B%22last_updated_date%22:%7B%7D%7D%7D,%7B%22match_phrase_prefix%22:%7B%22source.system%22:%22ellipsis%22%7D%7D%5D,%22must_not%22:%5B%7B%22match_phrase%22:%7B%22subtype%22:%22live-all%22%7D%7D,%7B%22match_phrase%22:%7B%22subtype%22:%22live-update%22%7D%7D,%7B%22match_phrase%22:%7B%22subtype%22:%22live-reporter-insight%22%7D%7D,%7B%22match_phrase%22:%7B%22subtype%22:%22live-race-call%22%7D%7D,%7B%22match_phrase%22:%7B%22subtype%22:%22live-faq%22%7D%7D,%7B%22match_phrase%22:%7B%22taxonomy.tags.slug%22:%22ellipsis-template%22%7D%7D%5D%7D%7D%7D&&_sourceInclude=headlines.basic,headlines.meta_title,headlines.url,canonical_url,content_elements.type,content_elements.content,additional_properties.commercial_node,description.basic,publish_date,slug,type,subtype,taxonomy.primary_section.path,taxonomy.primary_section._id,website_url



def get_prism_search_data_by_query(query):
  # &timeframe=1|day      <= not sure this works
  #https://prism.ext.nile.works/content/v4/search/published?q=+headlines.basic:"lewiston"+subtype:"default"&nocache=true&s=date&from=0&t=story&size=30&sort=display_date:desc&website=washpost&key={wapo_prism_key}
  # query_text = 'q=+headlines.basic:"'+ query +'" AND (subtype:default)'
  query_text = 'q=+headlines.basic:"'+ query +'" AND !(subtype:live-all) AND !(subtype:live-update) AND !(subtype:live-reporter-insight)'
  query_text += ' AND !(canonical_url:"")'
  query_text += ' AND (type:"story")'

  fullpath = 'https://prism.ext.nile.works/content/v4/search/published?{}&nocache=true&s=date&from=0&t=story&size=30&sort=display_date:desc&website=washpost&key={}'.format(
      query_text,
      wapo_prism_key
  )

  # +subtype:"default" # removes live
  # print(fullpath)
  response = requests.get(fullpath)
  my_json_object = json.loads(response.content)
  return my_json_object

def get_prism_search_linkboxes():
  # ?website=washpost
  # &sort=last_updated_date:desc
  # &from=0
  # &size=20
  # &body={
  #     "query":{
  #         "bool":{
  #             "must":[
  #                 {
  #                   "term":{
  #                     "type":"story"
  #                     }
  #                 },
  #                 {
  #                   "match_phrase_prefix":{
  #                     "slug":"linkbox"
  #                   }
  #                 },
  #                 {
  #                   "term":{
  #                     "revision.published":"true"
  #                   }
  #                 },
  #                 {
  #                   "range":{
  #                     "last_updated_date":{}
  #                   }
  #                 },
  #                 {
  #                   "match_phrase_prefix":{
  #                     "source.system":"ellipsis"
  #                   }
  #                 }
  #             ],
  #             "must_not":[
  #                 {"match_phrase":{"subtype":"live-all"}},
  #                 {"match_phrase":{"subtype":"live-update"}},
  #                 {"match_phrase":{"subtype":"live-reporter-insight"}},
  #                 {"match_phrase":{"subtype":"live-race-call"}},
  #                 {"match_phrase":{"subtype":"live-faq"}},
  #                 {"match_phrase":{"taxonomy.tags.slug":"ellipsis-template"}}
  #             ]
  #           }
  #       }
  #   }



  # ?website=washpost&sort=last_updated_date:desc&from=0&size=20&body={"query":{"bool":{"must":[{"term":{"type":"story"}},{"match_phrase_prefix":{"slug":"linkbox"}},{"term":{"revision.published":"true"}},{"range":{"last_updated_date":{}}},{"match_phrase_prefix":{"source.system":"ellipsis"}}],"must_not":[{"match_phrase":{"subtype":"live-all"}},{"match_phrase":{"subtype":"live-update"}},{"match_phrase":{"subtype":"live-reporter-insight"}},{"match_phrase":{"subtype":"live-race-call"}},{"match_phrase":{"subtype":"live-faq"}},{"match_phrase":{"taxonomy.tags.slug":"ellipsis-template"}}]}}}
  # website=washpost&sort=last_updated_date:desc&from=0&size=20&body={"query":{"bool":{"must":[{"term":{"type":"story"}},{"term":{"revision.published":"true"}},{"range":{"last_updated_date":{}}},{"match_phrase_prefix":{"source.system":"ellipsis"}},{"match_phrase":{"subtype":"link-box"}}],"must_not":[{"match_phrase":{"subtype":"live-all"}},{"match_phrase":{"subtype":"live-update"}},{"match_phrase":{"subtype":"live-reporter-insight"}},{"match_phrase":{"subtype":"live-race-call"}},{"match_phrase":{"subtype":"live-faq"}},{"match_phrase":{"taxonomy.tags.slug":"ellipsis-template"}}]}}}
  query_text = 'https://prism.ext.nile.works/content/v4/search/?website=washpost&sort=last_updated_date:desc&from=0&size=20'
  query_text += '&body={"query":{"bool":{"must":[{"term":{"type":"story"}},{"match_phrase_prefix":{"slug":"linkbox"}},{"term":{"revision.published":"true"}},'
  query_text += '{"range":{"last_updated_date":{}}},{"match_phrase_prefix":{"source.system":"ellipsis"}}],"must_not":[{"match_phrase":{"subtype":"live-all"}},'
  query_text += '{"match_phrase":{"subtype":"live-update"}},{"match_phrase":{"subtype":"live-reporter-insight"}},{"match_phrase":{"subtype":"live-race-call"}},'
  query_text += '{"match_phrase":{"subtype":"live-faq"}},{"match_phrase":{"taxonomy.tags.slug":"ellipsis-template"}}]}}}'
  # query_text += ' AND !(canonical_url:"")'
  # query_text += ' AND (type:"story") AND (subtype:"link-box")'
  query_text += f"&key={wapo_prism_key}"
  query_text += '&_sourceInclude=headlines.basic,headlines.meta_title,headlines.url,canonical_url,content_elements.type,content_elements.content,additional_properties.commercial_node,description.basic,publish_date,slug,type,subtype,taxonomy.primary_section.path,taxonomy.primary_section._id,website_url'

  # fullpath = 'https://prism.ext.nile.works/content/v4/search/published?{}&nocache=true&s=date&from=0&t=story&size=30&sort=display_date:desc&website=washpost&key={}'.format(
  #     query_text,
  #     wapo_prism_key
  # )

  # +subtype:"default" # removes live
  # print(fullpath)
  response = requests.get(query_text)
  my_json_object = json.loads(response.content)
  return my_json_object



In [ ]:
# @title 3rd party API functions (non-AI: content_API, ReCirc tool, etc)


def get_content_from_content_elements(content_elements, headline = '', keep_html = False):
  full_story = [] #[headline] if headline != '' else []
  for element in content_elements:
    if element['type'] == 'text' and "content" in element:
      # print( element['content'])
      full_story.append(element['content'])

  soup = BeautifulSoup("".join(str(x) for x in full_story))
  full_text = soup.get_text() if keep_html != True else soup.html

  return full_text

# def get_full_story_from_search_results(search_object, add_headline = True):

def print_html_search_results(the_search_object):
  xString = ''
  for story in the_search_object['content_elements']:
    # print(get_story_headline( story ))
    xString += '<h2>' + get_story_headline( story ) + "</h2>"
    if "content_elements" in story:
      for item in story['content_elements']:
        if "content" in item:
          # print('-- ' + item['content'])
          xString += '<P>'+ item['content'] + "</P>"
  return IPython.display.HTML(xString)

def print_html_search_results_headlines(the_search_object):
  xString = ''
  for story in the_search_object['content_elements']:
    # print(get_story_headline( story ))
    xString += '<h2>' + get_story_headline( story ) + "</h2>"
  return IPython.display.HTML(xString)


def get_full_story_story_object(story_object, add_headline = True):
  # print( 'get_full_story_story_object' )
  # print( story_object )

  headline = get_story_headline( story_object )+'. ' if add_headline else ''
  return get_content_from_content_elements(get_content_elements(story_object), headline, True)

def get_full_story_html_api(story_object, add_headline = True ):
  headline = '<h1>' + get_story_headline( story_object ) + '</h1>' if add_headline else ''
  return get_content_from_content_elements(get_content_elements(story_object), headline, True)



# Testing

In [ ]:
# @title Testing search Object

# search_object = get_prism_search_data_by_query("Taylor Swift")
search_object = get_prism_search_linkboxes()

print_html_search_results(search_object)


In [ ]:
# @title AI SUMMARIZE
# messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})

theResponse = ''
def ai_get_summarize_story(this_story_object):
  global theResponse

  system_prompt = """You are a journalism news editor and writer.
  Please summarize the provided article into a list of bullet points. Each bullet point should convey a key point from the story.
  It is critical that you maintain any original relevant hyperlinks within the text of the bullet points.
  Please follow these instructions carefully:
    1. Present an html bullet-point summary, ensuring each point is clear and concise.
    2. Embed washingtonpost.com hyperlinks directly into the summary points, around the specific word(s) the link relates to. Not at end of bullet-points.
    3. Make sure each bullet point captures a single key fact or idea.
    4. Do not add any additional information or context not found in the original story.
    5. Include any important quotes from the original story.
  Please pay close attention to the integrity of the hyperlinks and their placement within the summary to reflect how they are used in context.
  "."""

  prompt = """
  Headline: {}
  Story:
  {}

  Please summarize the provided article into a list of bullet points. Each bullet point should convey a key point from the story.
  It is critical that you maintain any original relevant hyperlinks within the text of the bullet points.
  Please follow these instructions carefully:
    1. Present an html bullet-point summary, ensuring each point is clear and concise.
    2. Embed washingtonpost.com hyperlinks directly into the summary points, around the specific word(s) the link relates to. Not at end of bullet-points.
    3. Make sure each bullet point captures a single key fact or idea.
    4. Do not add any additional information or context not found in the original story.
    5. Include any important quotes from the original story.
  Please pay close attention to the integrity of the hyperlinks and their placement within the summary to reflect how they are used in context.
  ".""".format(
      get_story_headline(this_story_object),
      get_full_story_html_api(this_story_object)
  )

  try:
    response = openai.chat.completions.create(
        # model="gpt-3.5-turbo",
        # model="gpt-4",
        # model="gpt-3.5-turbo-16k",
        model="gpt-4-1106-preview",
        # model=open_ai_model,
        # response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        # functions = seo_custom_functions,
        # function_call = 'auto',
        temperature=0.0,
    )
    theResponse = response    # this is for testing
    return theResponse.choices[0].message.content

  except openai.APIError as e:
    print(f"----ERROR: OpenAI API returned an API Error: {e}")
    return { 'error': e }
    pass

  return {}

#Summarize the story above. Pulling out the most important content.
  # List each embedded link in story, with an appropriate text title for it.
  # Any important quotes should be kept intact and added to the summary.
#    - list hyperlinks from news story (washingtonpost.com ONLY)

In [ ]:
# print( get_story_headline( story_object ))
response = ai_get_summarize_story(story_object)
print(response)
print(len(response))

full_story_html = get_full_story_html_api(story_object)
print(len(str(full_story_html)))

def get_heading_tags(soup):
    headings = {}
    for tag in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
        headings[tag] = [heading.text for heading in soup.find_all(tag)]
    return headings

print( get_heading_tags(full_story_html))

<ul>
<li>Sierra Edmisten is collecting signatures in Nebraska for a ballot initiative to require employers to provide at least five days of paid sick leave annually.</li>
<li>Previous attempts to pass similar paid sick leave laws have failed in the Nebraska legislature and U.S. Congress, despite <a href="https://www.cbsnews.com/news/poll-who-can-get-ahead-in-the-u-s/" target="_blank">polls</a> indicating widespread support.</li>
<li>Congress and state legislatures are often gridlocked or dominated by one party, prompting citizens in <a href="https://www.ncsl.org/elections-and-campaigns/initiative-and-referendum-states" target="_blank">states with referendum capabilities</a> to pursue direct democracy through ballot initiatives.</li>
<li>Ballot initiatives have been used to pass significant policies, such as raising the minimum wage and legalizing marijuana, and have recently been a battleground for abortion rights in various states.</li>
<li>Direct democracy is seen as imperfect due to

In [ ]:
# @title Create LinkBox

  # fullpath = 'http://api.washingtponst.com/v5/related_stories/api/?q='+ orig_story_url
  # response = requests.get(fullpath)
  # my_json_object = json.loads(response.content)
  # return my_json_object

# def combine_results_content_into_one(this_story_list, max_num = 10):


def create_linkbox(this_story_object):
    # global story_object
    print("Creating Linkbox for story: "+ get_story_headline( this_story_object ))
    xtags = get_tags(this_story_object)
    print("tags: " + ','.join(xtags))
    similar_stories = get_prism_stories_by_tag(xtags[0])


    headlines = [story['headlines']['basic'] for story in similar_stories['content_elements']]

    xString = ''
    for story in the_search_object['content_elements']:
      # print(get_story_headline( story ))
      xString += '<h2>' + get_story_headline( story ) + "</h2>"
      if "content_elements" in story:
        for item in story['content_elements']:
          if "content" in item:
            # print('-- ' + item['content'])
            xString += '<P>'+ item['content'] + "</P>"
    return IPython.display.HTML(xString)

    # print(headlines)
    # summaries = generate_summary(headlines, openai_api_key)

    # This will combine the headlines, summaries, and links into the final HTML format
    # linkbox_html = '<h2>Related Articles</h2>'
    # for headline, summary in zip(headlines, summaries['choices'][0]['text'].split('\n\n')):
    #     link = "link not passed from PRISM"
    #     linkbox_html += f"<p><b>{headline['basic']}:</b> {summary} <a href=\"{link}\" target=\"_blank\">Read more</a>.</p>"

    # return linkbox_html
    return "bing"

try:
    linkbox = create_linkbox(story_object)
    print(linkbox)
except Exception as e:
    print(str(e))

In [ ]:
# @title Testing Story Object
# story_object = get_prism_data_from_url("https://www.washingtonpost.com/world/2023/11/14/israel-war-hamas-gaza-al-shifa-news/")
# story_object = get_prism_data_from_url("https://www.washingtonpost.com/world/2023/11/15/us-hostages-israel-gaza-war/")
# story_object = get_prism_data_from_url("https://www.washingtonpost.com/national-security/2023/11/15/hamas-israel-gaza-hostage-deal/")
# story_object = get_prism_data_from_url("https://www.washingtonpost.com/politics/2023/11/15/israel-palestinians-gaza-aid/")
# story_object = get_prism_data_from_url("https://www.washingtonpost.com/politics/2023/11/15/biden-xi-meeting/")
# story_object = get_prism_data_from_url("https://www.washingtonpost.com/politics/2023/11/15/nebraska-ballot-initiative-paid-sick-leave/")

# print(get_full_story_html_api( story_object ))
# print(story_object['taxonomy']['tags'])
print(get_story_headline( story_object ))

xtags = get_tags(story_object)
print(xtags)

print ( get_full_story_html_api(story_object) )
# print( get_tags(story_object) )




# similar_stories = get_prism_stories_by_tag(xtags[0])
# similar_stories

# print_html_search_results(similar_stories)

# print_html_search_results_headlines(similar_stories)


A boom of ballot initiatives is reshaping this state’s democracy 
['democracyinamerica']
<html><body><p>CRETE, Neb. — This is what the future of American democracy looked like for Sierra Edmisten one evening this fall: a block of modest, single-story homes; the lights from the Crete High School football stadium glowing in the mist; families gathering for dinner after a day in the meatpacking plants.Edmisten was collecting signatures for a cause associated with liberals in a red part of one of the country’s reddest states.<b> </b>The ballot initiative she was seeking to put before voters would, if passed, require Nebraska employers to guarantee their workers at least five days of paid sick leave each year.A push to enact a similar law had failed twice in the Nebraska legislature and repeatedly since 2016 in the U.S. Congress,<b> </b>despite <a href="https://www.cbsnews.com/news/poll-who-can-get-ahead-in-the-u-s/" target="_blank">polls</a> suggesting widespread <a href="https://www.prri.

In [ ]:
# @title AI get questions from stories
# messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})


def ai_get_questions_from_story(this_story_object):
  prompt = """
  Headline: {}
  Story:
  {}

  From the story above, please identify appropriate section headings under which the story should be categorized for publication.
  Hierarchy of headings in order (e.g., primary, secondary)
  For example if the story was about game 3 of the world series, the section titles would be "World Series", "Game 3" in that order.
  ".""".format(
      get_story_headline(this_story_object),
      get_full_story_html_api(this_story_object)
  )
  # print( prompt );
  # return prompt
  # Examples of this would be "Israel-Gaza war", "Wildfires in Hawaii", "The Mass shooting in Maine", "Trump Jan. 6 indictment".
  try:
    response = openai.chat.completions.create(
        # model="gpt-3.5-turbo",
        # model="gpt-4",
        # model="gpt-3.5-turbo-16k",
        model="gpt-4-1106-preview",
        # model=open_ai_model,
        messages=[
            {"role": "system", "content": "You are a journalism news editor and writer."},
            {"role": "user", "content": prompt}
        ],
        # functions = seo_custom_functions,
        # function_call = 'auto',
        temperature=0.0,
        # max_tokens=open_ai_max_tokens,


    )
    # response_json = json.loads(response.choices[0].message.function_call.arguments)

    # print('------------AI response')
    # print(response)
    # response_json = json.loads(response.choices[0].message.function_call.arguments)
    # print(response_json)
    # response_keywords = response_json['keywords'].split(', ')
    # response_meta_descriptions = response_json['meta_descriptions'].split('**')

    # response_json['keywords'] = [s.strip() for s in response_keywords]
    # response_json['meta_descriptions'] = [s.strip() for s in response_meta_descriptions]

    return response

  except openai.APIError as e:
    #Handle API error here, e.g. retry or log
    print(f"----ERROR: OpenAI API returned an API Error: {e}")
    # return json of error
    return { 'error': e }
    pass

  return {}


